In [ ]:
import torch
from torch import nn , optim 
import torch.nn.functional as F 

    

In [ ]:
data_dir = 'dataset/Cat_Dog_data'
a
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()]) 

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256), #256*256
                                      transforms.ToTensor()])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train_okey', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test_ok', transform=test_transforms)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)


In [ ]:
# change this to the trainloader or testloader 
data_iter = iter(trainloader)

images, labels = next(data_iter)
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
for ii in range(4):
    ax = axes[ii]
    helper.imshow(images[ii], ax=ax, normalize=False)

## classifier CNN class  

In [ ]:
#creating the cnn class which was going to used in cats_dogs
class CNN(nn.Module) :
    def __init__(self):
        super().__init__()

        #defining no of convo layers to detect features 
        #image = 256*256
        self.conv1 = nn.Conv2d(3 , 16 , 3 , padding=1 )
        #image is half bcz of pooling ... 128*128*16
        self.conv2 = nn.Conv2d(16 , 32 , 3 , padding=1 )
        # #image is half bcz of pooling ... 64*64*32
        self.conv3 = nn.Conv2d(32 , 64 , 3 , padding=1 )
        #32*32*64
        
        #pooling 
        self.pool2d = nn.MaxPool2d(2,2)
        
        #now lets start the fullyConnected linear to predict the values 
        self.fc1 = nn.Linear(32*32*64  , 256 ) #input
        self.fc2 = nn.Linear(256 , 128 )
        self.fc3 = nn.Linear(128 , 64 )
        self.fc4 = nn.Linear(64 , 2 )#output
        
        #dropOut bcz of overfitting 
        self.Dropout = nn.Dropout(0.2)
        
    def forward(self, x ) :
        #conv pr bhe activationFunction lage gein 
        x = self.pool2d(  F.relu( self.conv1(x) ) ) 
        x = self.pool2d(  F.relu( self.conv2(x) ) ) 
        x = self.pool2d(  F.relu( self.conv3(x) ) ) 
        
        #change the image size :: flattern 
        
        x = x.view(-1 , 32*32*64) 
        
        #now predicting Linear 
        x =  F.relu( self.fc1(x) ) 
        x =  F.relu( self.fc2(x) )   
        x =  F.relu( self.fc3(x) )  
        x =  F.log_softmax( self.fc4(x) ,dim=1)  
        return x
        
        
        

In [ ]:
model = CNN()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)




## Training 

In [ ]:
epochs = 20

train_losses, test_losses = [], []
accuracy_tol = []
for e in range(epochs):
    tot_train_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
       
        log_ps = model.forward(images)
        print("e=",e)
        loss = criterion(log_ps, labels)
        tot_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    else:
        tot_test_loss = 0
        test_correct = 0  # Number of correct predictions on the test set
        
        # # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                loss = criterion(log_ps, labels)
                tot_test_loss += loss.item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                test_correct += equals.sum().item()

        # Get mean loss to enable comparison between train and test sets
        train_loss = tot_train_loss / len(trainloader.dataset)
        test_loss = tot_test_loss / len(testloader.dataset)
        accuracy = test_correct / len(testloader.dataset) 
        # At completion of epoch
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracy_tol.append(accuracy)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_loss),
              "accuracy" , accuracy 
              )

## anaylsis train loss and test loss using grpah 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='test_loss loss')
plt.plot(accuracy_tol, label='Accuracy test')

plt.legend(frameon=False)

## predicting 

In [ ]:


dog_probs = []
model.eval()
with torch.no_grad():
    for data, fileid in testloader:
        # print(data[0].shape)
        # print(data[0].unsqueeze(0).shape)
        #data = data.to(device)
        preds = model.forward(data[0].unsqueeze(0))
        
        preds_list = F.softmax(preds, dim=1)[:].tolist()
        print(preds_list)
        helper.imshow(data[0], normalize=False)
        dog_probs += list(zip(list(fileid), preds_list))
        
        




In [ ]:
dog_probs.sort(key = lambda x : int(x[0]))
dog_probs